In [1]:
import os, sys
import numpy as np

import xarray as xr
from matplotlib import pyplot as plt
%matplotlib inline

#from mitequinox.utils import *
from mitequinox.binary import *

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/utils.py:137: RuntimeWarning: Couldn't detect a suitable IP address for reaching '8.8.8.8', defaulting to '127.0.0.1': [Errno 101] Network is unreachable
  RuntimeWarning,


In [2]:
from dask.distributed import Client, LocalCluster
#
cluster = LocalCluster()
#
#from dask_jobqueue import PBSCluster
#cluster = PBSCluster()
#w = cluster.scale(28*1)
#
client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:54278 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 56 Memory: 107.37 GB


In [3]:
out_dir = osi+'equinox/mit4320/zarr/'
out_dir

'/home/datawork-lops-osi/equinox/mit4320/zarr/'

In [5]:
# zarr compression
# http://xarray.pydata.org/en/stable/io.html
# http://zarr.readthedocs.io/en/stable/tutorial.html#compressors
#compressor = None

---

# store grid

In [4]:
ds_index, ds = get_compressed_level_index(grid_dir)

# we will need to keep some of these when computing gradients
#ds = ds.drop(['dxG','dyG','dxC','dyC','rAw','rAs','rAz'])
#ds = ds.drop(['hFacC','hFacW','hFacS'])
#ds = ds.drop(['maskC','maskW','maskS'])
#ds = ds.drop(['Z', 'Zp1', 'Zu', 'Zl', 'drC', 'drF','PHrefC','PHrefF'])

#Nc = 432 # original choice
#Nc = 27 # very long scheduling
#Nc = 96 # 96*45
#ds = ds.chunk({'i': Nc, 'j': Nc, 'i_g': Nc, 'j_g': Nc})

print(ds)

/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/xmitgcm/mds_store.py:837: UserWarning: Couldn't find available_diagnostics.log in  or /home/datawork-lops-osi/equinox/mit4320/grid/. Using default version.
  "in %s or %s. Using default version." % (data_dir, grid_dir))


<xarray.Dataset>
Dimensions:  (face: 13, i: 4320, i_g: 4320, j: 4320, j_g: 4320, k: 90, k_l: 90, k_p1: 91, k_u: 90)
Coordinates:
  * i        (i) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * i_g      (i_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j        (j) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * j_g      (j_g) int64 0 1 2 3 4 5 6 7 ... 4313 4314 4315 4316 4317 4318 4319
  * k        (k) int64 0 1 2 3 4 5 6 7 8 9 10 ... 80 81 82 83 84 85 86 87 88 89
  * k_u      (k_u) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k_l      (k_l) int64 0 1 2 3 4 5 6 7 8 9 ... 80 81 82 83 84 85 86 87 88 89
  * k_p1     (k_p1) int64 0 1 2 3 4 5 6 7 8 9 ... 81 82 83 84 85 86 87 88 89 90
  * face     (face) int64 0 1 2 3 4 5 6 7 8 9 10 11 12
    XC       (face, j, i) >f4 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    YC       (face, j, i) >f4 dask.array<chunksize=(1, 4320, 4320), meta=np.ndarray>
    XG      

In [5]:
# this way too slow, this may be optimized at xmitgcm calls
ds = ds.isel(k=0).persist()
#ds.rAw.isel(face=1).plot()

In [ ]:
file_out = out_dir+'grid.zarr'
%time ds.to_zarr(file_out, mode='w')
file_out

--- 

# !!! do not look beyond this point !!!

# standard data layout: chunks (face, time, j, i) = (1, 1, 4320, 4320)

In [ ]:
#V = ['Eta', 'SST', 'SSS', 'SSU', 'SSV']
V = ['oceTAUX']
#V = ['oceTAUX', 'oceTAUY', 'KPPhbl']


# scheduler does not like (i.e. takes a long time to do anything) any spatial rechunking on top 
# of that on faces:
#Nc = 480 # x 9 = 4320
#Nc = 96 # x 45 = 4320
# other choices: 432, 27 (long scheduling), 288, 864

#out_dir = scratch+'/mit/standard/'
#scratchd = '/home/c11-data/Test_aponte/'
#out_dir = scratchd+'/mit/standard/'

for v in V:
    #
    data_dir = root_data_dir+v+'/'
    # !!! should be removed eventually
    data_dir = root_data_dir+'tmp/'
    # !!!    
    iters, time = get_iters_time(v, data_dir, delta_t=25.)
    #
    p = 'C'
    if v in ['SSU','oceTAUX']:
        p = 'W'
    elif v is ['SSV','oceTAUY']:
        p = 'S'
    #
    ds = get_compressed_data(v, data_dir, grid_dir, iters=iters, time=time, client=client, point=p)
    #
    # should store grid data independantly in a single file
    ds = ds.drop(['XC','YC','Depth','rA'])
    #
    #ds = ds.isel(time=slice(1000))
    #ds = ds.chunk({'face': 1})
    #ds = ds.chunk({'face': 1, 'i': Nc, 'j': Nc}) # scheduler does not like this
    #
    dv = ds[v].to_dataset()
    #
    #dv = dv.chunk({'i': Nc, 'j': Nc}) # scheduler does not like this either
    #dv = dv.isel(time=0)
    #
    file_out = out_dir+'%s.zarr'%(v)
    try:
        #print(dv)
        %time dv.to_zarr(file_out, mode='w')                    
    except:
        print('Failure')
    dsize = getsize(file_out)
    print('   data is %.1fGB ' %(dsize/1e9))

---
# try to load standard lay out, rechunk and store right away


In [10]:
# parameters for rechunking

Nt = 24*10 # time chunks
#Nt = 0
#
Nt = len(ds.time) if Nt == 0 else Nt

Nc = 96 # x 45 = 4320
# other choices: 432, 27 (long scheduling), 288, 864

### one face at a time, all variables

In [12]:
# same but over all variables and faces
#V = ['SSU', 'SSV']
V = ['SSU']

out_dir = scratch+'mit/rechunked/'

for v in V:

    file_in = scratch+'/mit/standard/%s.zarr'%(v)
    ds0 = xr.open_zarr(file_in)
    
    #for face in range(ds0['face'].size):
    for face in [1]:
        
        ds = ds0.isel(face=face)
        #
        ds = ds.isel(time=slice(len(ds.time)//Nt *Nt))
        #
        ds = ds.chunk({'time': Nt, 'i': Nc, 'j': Nc})
        #
        # tmp, xarray zarr backend bug: 
        # https://github.com/pydata/xarray/issues/2278
        del ds['face'].encoding['chunks']
        del ds[v].encoding['chunks']
        
        file_out = out_dir+'%s_f%02d.zarr'%(v,face)
        try:
            #%time ds.to_zarr(file_out, mode='w')
            # specify compression:
            %time ds.to_zarr(file_out, mode='w', \
                             encoding={key: {'compressor': compressor} for key in ds.variables})
            # without compression: 601G for face 1
        except:
            print('Failure')
        dsize = getsize(file_out)
        print(' %s face=%d  data is %.1fGB ' %(v, face, dsize/1e9))


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 8min 24s, sys: 23.9 s, total: 8min 48s
Wall time: 14min 49s
 SSU face=1  data is 628.6GB 


---

In [17]:
w = cluster.start_workers(10)

In [6]:
client.restart()

distributed.scheduler - ERROR - Not all workers responded positively: ['timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out', 'timed out']
NoneType: None
distributed.client - ERROR - Restart timed out after 20.000000 seconds


Client Scheduler: tcp://10.148.1.12:42619 Dashboard: http://10.148.1.12:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


distributed.scheduler - ERROR - '1846309'
Traceback (most recent call last):
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/scheduler.py", line 1306, in add_worker
    plugin.add_worker(scheduler=self, worker=address)
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask_jobqueue/core.py", line 62, in add_worker
    self.running_jobs[job_id] = self.pending_jobs.pop(job_id)
KeyError: '1846309'
distributed.scheduler - ERROR - '1846313'
Traceback (most recent call last):
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/distributed/scheduler.py", line 1306, in add_worker
    plugin.add_worker(scheduler=self, worker=address)
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.6/site-packages/dask_jobqueue/core.py", line 62, in add_worker
    self.running_jobs[job_id] = self.pending_jobs.pop(job_id)
KeyError: '1846313'
distributed.scheduler - ERROR - '1846313

In [13]:
# kill scheduler, workers
cluster.close()
#cluster.stop_workers(cluster.jobs)

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/client.py", line 1283, in _close
    await gen.with_timeout(timedelta(seconds=2), list(coroutines))
concurrent.futures._base.CancelledError
distributed.utils - ERROR - 
Traceback (most recent call last):
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/utils.py", line 666, in log_errors
    yield
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/client.py", line 1012, in _reconnect
    await self._close()
  File "/home1/datahome/aponte/.miniconda3/envs/equinox/lib/python3.7/site-packages/distributed/cli